In [1]:
!pip install -q --upgrade diffusers transformers accelerate scipy ftfy Pillow opencv-python gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 54.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 110.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 69.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 32.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 102.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:0

In [2]:
import torch
import requests
from PIL import Image
from io import BytesIO
import gradio as gr
import numpy as np
import cv2
from diffusers import (
    StableDiffusionPipeline,
    StableDiffusionImg2ImgPipeline,
    StableDiffusionInpaintPipeline,
    StableDiffusionControlNetPipeline,
    ControlNetModel,
    UniPCMultistepScheduler,
    DDIMScheduler,
    EulerAncestralDiscreteScheduler,
    DPMSolverMultistepScheduler
)
import gc

2025-04-28 06:33:20.517626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745822000.712988      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745822000.771858      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

print(f"Using device: {device}")
print(f"Using dtype: {torch_dtype}")

Using device: cuda
Using dtype: torch.float16


In [4]:
def load_t2i_pipe(model_id="runwayml/stable-diffusion-v1-5"):
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch_dtype)
    pipe = pipe.to(device)
    return pipe

def load_i2i_pipe(model_id="runwayml/stable-diffusion-v1-5"):
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch_dtype)
    pipe = pipe.to(device)
    return pipe

def load_inpaint_pipe(model_id="runwayml/stable-diffusion-v1-5"):
    inpaint_model_id = "runwayml/stable-diffusion-inpainting"
    try:
        pipe = StableDiffusionInpaintPipeline.from_pretrained(inpaint_model_id, torch_dtype=torch_dtype)
    except Exception as e:
        print(f"Inpainting model yüklenemedi ({e}), standart model kullanılıyor.")
        pipe = StableDiffusionInpaintPipeline.from_pretrained(model_id, torch_dtype=torch_dtype)
    pipe = pipe.to(device)
    return pipe

def load_controlnet_pipe(model_id="runwayml/stable-diffusion-v1-5", controlnet_model_id="lllyasviel/sd-controlnet-canny"):
    controlnet = ControlNetModel.from_pretrained(controlnet_model_id, torch_dtype=torch_dtype)
    pipe = StableDiffusionControlNetPipeline.from_pretrained(
        model_id, controlnet=controlnet, torch_dtype=torch_dtype
    )
    pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
    pipe = pipe.to(device)
    return pipe

print("Loading pipelines...")
try:
    pipe_t2i = load_t2i_pipe()
    pipe_i2i = load_i2i_pipe()
    pipe_inpaint = load_inpaint_pipe()
    pipe_controlnet_canny = load_controlnet_pipe()
    print("All pipelines loaded successfully.")
    PIPELINES_LOADED = True
except Exception as e:
    print(f"Error loading all pipelines upfront: {e}")
    print("Pipelines will be loaded on demand (slower).")
    PIPELINES_LOADED = False
    pipe_t2i, pipe_i2i, pipe_inpaint, pipe_controlnet_canny = None, None, None, None

def cleanup_memory():
    """Belleği temizlemeye çalışır."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("Memory cleanup attempted.")

schedulers = {
    "Default (PNDM)": None, 
    "DDIM": DDIMScheduler,
    "Euler Ancestral": EulerAncestralDiscreteScheduler,
    "DPM++ 2M Karras": DPMSolverMultistepScheduler 
}

def set_scheduler(pipe, scheduler_name):
    scheduler_class = schedulers.get(scheduler_name)
    if pipe is not None and scheduler_class is not None:
        print(f"Setting scheduler to {scheduler_name}")
        pipe.scheduler = scheduler_class.from_config(pipe.scheduler.config)
    elif pipe is not None:
        print(f"Setting scheduler to Default")
        pass 
    return pipe

Loading pipelines...


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

All pipelines loaded successfully.


In [5]:
def generate_i2i(init_image_pil, prompt, negative_prompt, strength, num_steps, guidance_scale, scheduler_name):
    global pipe_i2i
    if init_image_pil is None:
        return None, "Please upload an initial image."

    if not PIPELINES_LOADED or pipe_i2i is None:
        print("Loading I2I pipeline on demand...")
        pipe_i2i = load_i2i_pipe()
        cleanup_memory()
    if pipe_i2i is None: return None, "I2I Pipeline yüklenemedi."

    pipe_i2i = set_scheduler(pipe_i2i, scheduler_name)

    target_size = (512, 512)
    init_image_pil = init_image_pil.resize(target_size)

    try:
        print(f"Generating I2I image for prompt: {prompt}")
        with torch.autocast("cuda" if device == "cuda" else "cpu"):
            image = pipe_i2i(
                prompt=prompt,
                negative_prompt=negative_prompt,
                image=init_image_pil,
                strength=strength,
                num_inference_steps=int(num_steps),
                guidance_scale=guidance_scale
            ).images[0]
        print("I2I image generated.")
        return image, "Success"
    except Exception as e:
        print(f"I2I Error: {e}")
        cleanup_memory()
        return None, f"Error: {e}"

def generate_controlnet_canny(control_image_pil, prompt, negative_prompt, num_steps, guidance_scale, canny_low, canny_high):
    global pipe_controlnet_canny
    if control_image_pil is None:
        return None, None, "Please upload an image to extract edges."

    if not PIPELINES_LOADED or pipe_controlnet_canny is None:
        print("Loading ControlNet Canny pipeline on demand...")
        pipe_controlnet_canny = load_controlnet_pipe()
        cleanup_memory()

    if pipe_controlnet_canny is None: return None, None, "ControlNet Canny Pipeline yüklenemedi."

    control_image_np = np.array(control_image_pil)

    h, w, _ = control_image_np.shape
    if max(h, w) > 600:
         scale = 600 / max(h, w)
         new_w, new_h = int(w*scale), int(h*scale)
         control_image_np = cv2.resize(control_image_np, (new_w, new_h))
         control_image_pil = Image.fromarray(control_image_np) # PIL'i de güncelle

    canny_map_np = cv2.Canny(control_image_np, canny_low, canny_high)
    canny_map_pil = Image.fromarray(cv2.cvtColor(canny_map_np, cv2.COLOR_GRAY2BGR))

    try:
        print(f"Generating ControlNet Canny image for prompt: {prompt}")
        with torch.autocast("cuda" if device == "cuda" else "cpu"):
             image = pipe_controlnet_canny(
                 prompt=prompt,
                 negative_prompt=negative_prompt,
                 image=canny_map_pil,
                 num_inference_steps=int(num_steps),
                 guidance_scale=guidance_scale
             ).images[0]
        print("ControlNet Canny image generated.")
        return canny_map_pil, image, "Success"
    except Exception as e:
        print(f"ControlNet Canny Error: {e}")
        cleanup_memory()
        return None, None, f"Error: {e}"

In [6]:
def generate_t2i(prompt, negative_prompt, num_steps, guidance_scale, scheduler_name):
    global pipe_t2i
    if not PIPELINES_LOADED or pipe_t2i is None:
        print("Loading T2I pipeline on demand...")
        pipe_t2i = load_t2i_pipe()
        cleanup_memory()

    if pipe_t2i is None: return None, "T2I Pipeline yüklenemedi."

    pipe_t2i = set_scheduler(pipe_t2i, scheduler_name)

    try:
        print(f"Generating T2I image for prompt: {prompt}")
        with torch.autocast("cuda" if device == "cuda" else "cpu"):
             image = pipe_t2i(
                prompt=prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=int(num_steps),
                guidance_scale=guidance_scale
             ).images[0]
        print("T2I image generated.")
        return image, "Success"
    except Exception as e:
        print(f"T2I Error: {e}")
        cleanup_memory()
        return None, f"Error: {e}"
def generate_inpaint(image_dict, prompt, negative_prompt, num_steps, guidance_scale, scheduler_name):
    global pipe_inpaint
    if image_dict is None or "image" not in image_dict or "mask" not in image_dict:
         return None, "Please provide an image and draw a mask."

    init_image_pil = image_dict["image"]
    mask_image_pil = image_dict["mask"]

    if not PIPELINES_LOADED or pipe_inpaint is None:
        print("Loading Inpaint pipeline on demand...")
        pipe_inpaint = load_inpaint_pipe()
        cleanup_memory()

    if pipe_inpaint is None: return None, "Inpaint Pipeline yüklenemedi."

    pipe_inpaint = set_scheduler(pipe_inpaint, scheduler_name)

    target_size = (512, 512)
    init_image_pil = init_image_pil.resize(target_size)
    mask_image_pil = mask_image_pil.resize(target_size).convert("L")


    try:
        print(f"Generating Inpaint image for prompt: {prompt}")
        with torch.autocast("cuda" if device == "cuda" else "cpu"):
            image = pipe_inpaint(
                prompt=prompt,
                negative_prompt=negative_prompt,
                image=init_image_pil,
                mask_image=mask_image_pil,
                num_inference_steps=int(num_steps),
                guidance_scale=guidance_scale
            ).images[0]
        print("Inpaint image generated.")
        return image, "Success"
    except Exception as e:
        print(f"Inpaint Error: {e}")
        cleanup_memory()
        return None, f"Error: {e}"

In [7]:
loaded_lora = None

def load_lora_weights(lora_id_or_path):
    global pipe_t2i, loaded_lora
    if not pipe_t2i:
        return "T2I pipeline not loaded yet. Generate an image first."

    unload_lora_weights()

    if not lora_id_or_path:
        return "Please provide a LoRA ID (Hugging Face Hub) or local path."

    try:
        print(f"Loading LoRA: {lora_id_or_path}")
        if "/" in lora_id_or_path and not lora_id_or_path.startswith("/"):
             pipe_t2i.load_lora_weights(lora_id_or_path) 
        else:
             weight_name = "pytorch_lora_weights.safetensors"
             if lora_id_or_path.endswith(".bin"): weight_name="pytorch_lora_weights.bin"
             pipe_t2i.load_lora_weights(lora_id_or_path, weight_name=weight_name)

        loaded_lora = lora_id_or_path
        print(f"LoRA '{lora_id_or_path}' loaded successfully.")
        return f"LoRA '{lora_id_or_path}' loaded. Remember to use trigger words in prompt if needed."
    except Exception as e:
        print(f"Error loading LoRA: {e}")
        return f"Error loading LoRA: {e}"

def unload_lora_weights():
    global pipe_t2i, loaded_lora
    if pipe_t2i and loaded_lora:
        try:
            print(f"Unloading LoRA: {loaded_lora}")
            pipe_t2i.unload_lora_weights()
            print("LoRA unloaded.")
            status = f"LoRA '{loaded_lora}' unloaded."
            loaded_lora = None
            cleanup_memory()
            return status
        except Exception as e:
             print(f"Error unloading LoRA: {e}")
             return f"Error unloading LoRA: {e}"
    elif pipe_t2i:
        return "No LoRA currently loaded."
    else:
        return "T2I pipeline not loaded."

In [8]:
with gr.Blocks() as demo:
    gr.Markdown("# Interactive Stable Diffusion Playground")
    gr.Markdown("Kaggle üzerinde çalışan Diffusers tabanlı bir arayüz.")

    with gr.Tab("Text-to-Image"):
        with gr.Row():
            with gr.Column(scale=2):
                t2i_prompt = gr.Textbox(label="Prompt (Ne çizmek istersiniz?)")
                t2i_neg_prompt = gr.Textbox(label="Negative Prompt (Neleri istemezsiniz?)", value="low quality, blurry, deformed")
                t2i_scheduler = gr.Dropdown(list(schedulers.keys()), label="Scheduler", value="Default (PNDM)")
                with gr.Row():
                    t2i_steps = gr.Slider(minimum=10, maximum=100, step=1, value=30, label="Inference Steps")
                    t2i_guidance = gr.Slider(minimum=1, maximum=20, step=0.5, value=7.5, label="Guidance Scale")
                t2i_button = gr.Button("Generate Image", variant="primary")
            with gr.Column(scale=1):
                t2i_output = gr.Image(label="Generated Image")
                t2i_status = gr.Textbox(label="Status", interactive=False)

        t2i_button.click(
            generate_t2i,
            inputs=[t2i_prompt, t2i_neg_prompt, t2i_steps, t2i_guidance, t2i_scheduler],
            outputs=[t2i_output, t2i_status]
        )
    with gr.Tab("Image-to-Image"):
        with gr.Row():
            with gr.Column(scale=1):
                i2i_input_image = gr.Image(type="pil", label="Initial Image")
            with gr.Column(scale=1):
                i2i_output_image = gr.Image(label="Generated Image")
                i2i_status = gr.Textbox(label="Status", interactive=False)

        i2i_prompt = gr.Textbox(label="Prompt (Ne olmasını istersiniz?)")
        i2i_neg_prompt = gr.Textbox(label="Negative Prompt", value="low quality, blurry, deformed")
        i2i_scheduler = gr.Dropdown(list(schedulers.keys()), label="Scheduler", value="Default (PNDM)")
        with gr.Row():
            i2i_strength = gr.Slider(minimum=0.0, maximum=1.0, step=0.05, value=0.75, label="Strength (Değişim Miktarı)")
            i2i_steps = gr.Slider(minimum=10, maximum=100, step=1, value=30, label="Inference Steps")
            i2i_guidance = gr.Slider(minimum=1, maximum=20, step=0.5, value=7.5, label="Guidance Scale")
        i2i_button = gr.Button("Generate Image", variant="primary")

        i2i_button.click(
            generate_i2i,
            inputs=[i2i_input_image, i2i_prompt, i2i_neg_prompt, i2i_strength, i2i_steps, i2i_guidance, i2i_scheduler],
            outputs=[i2i_output_image, i2i_status]
        )
    with gr.Tab("Inpainting"):
        gr.Markdown("Upload an image and draw a mask over the area you want to change.")
        with gr.Row():
            inpaint_input_image = gr.Image(type="pil", label="Image & Mask", height=400)
            with gr.Column():
                inpaint_output_image = gr.Image(label="Generated Image", height=400)
                inpaint_status = gr.Textbox(label="Status", interactive=False)

        inpaint_prompt = gr.Textbox(label="Prompt (Maskelenen alana ne çizilsin?)")
        inpaint_neg_prompt = gr.Textbox(label="Negative Prompt", value="low quality, blurry, deformed")
        inpaint_scheduler = gr.Dropdown(list(schedulers.keys()), label="Scheduler", value="Default (PNDM)")
        with gr.Row():
            inpaint_steps = gr.Slider(minimum=10, maximum=100, step=1, value=50, label="Inference Steps")
            inpaint_guidance = gr.Slider(minimum=1, maximum=20, step=0.5, value=8.0, label="Guidance Scale")
        inpaint_button = gr.Button("Generate Inpainted Image", variant="primary")

        inpaint_button.click(
            generate_inpaint,
            inputs=[inpaint_input_image, inpaint_prompt, inpaint_neg_prompt, inpaint_steps, inpaint_guidance, inpaint_scheduler],
            outputs=[inpaint_output_image, inpaint_status]
        )
    with gr.Tab("ControlNet (Canny Edge)"):
        gr.Markdown("Upload an image. Its edges will be detected (Canny) and used to guide the generation process based on your prompt.")
        with gr.Row():
            with gr.Column(scale=1):
                cn_input_image = gr.Image(type="pil", label="Input Image for Edge Detection")
                with gr.Row():
                   canny_low_thr = gr.Slider(minimum=1, maximum=255, step=1, value=100, label="Canny Low Threshold")
                   canny_high_thr = gr.Slider(minimum=1, maximum=255, step=1, value=200, label="Canny High Threshold")
            with gr.Column(scale=1):
                 cn_canny_map = gr.Image(label="Detected Canny Edges")
            with gr.Column(scale=1):
                cn_output_image = gr.Image(label="Generated Image Guided by Edges")
                cn_status = gr.Textbox(label="Status", interactive=False)

        cn_prompt = gr.Textbox(label="Prompt (Kenarlara uygun ne çizilsin?)")
        cn_neg_prompt = gr.Textbox(label="Negative Prompt", value="low quality, blurry, deformed, text, watermark")
        with gr.Row():
            cn_steps = gr.Slider(minimum=10, maximum=100, step=1, value=25, label="Inference Steps")
            cn_guidance = gr.Slider(minimum=1, maximum=20, step=0.5, value=7.5, label="Guidance Scale")
        cn_button = gr.Button("Generate with ControlNet", variant="primary")

        cn_button.click(
            generate_controlnet_canny,
            inputs=[cn_input_image, cn_prompt, cn_neg_prompt, cn_steps, cn_guidance, canny_low_thr, canny_high_thr],
            outputs=[cn_canny_map, cn_output_image, cn_status]
        )
    with gr.Tab("LoRA Loader (T2I Only)"):
        gr.Markdown("Load custom styles or concepts using LoRA weights. Enter Hugging Face Hub ID (e.g., `ostris/ikea-instructions-lora-sdv1-5`) or a local path in Kaggle (e.g., `/kaggle/input/my-lora/lora.safetensors`). Affects Text-to-Image tab only for now.")
        with gr.Row():
             lora_input = gr.Textbox(label="LoRA Hub ID or Local Path")
             lora_load_btn = gr.Button("Load LoRA")
             lora_unload_btn = gr.Button("Unload LoRA")
        lora_status = gr.Textbox(label="LoRA Status", interactive=False)

        lora_load_btn.click(load_lora_weights, inputs=[lora_input], outputs=[lora_status])
        lora_unload_btn.click(unload_lora_weights, inputs=[], outputs=[lora_status])


In [9]:
demo.close()

In [ ]:
demo.launch(debug=True, share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://7d3f23f8814a339fd3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Loading LoRA: /kaggle/input/lora-tensors/pytorch_lora_weights.safetensors


No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


LoRA '/kaggle/input/lora-tensors/pytorch_lora_weights.safetensors' loaded successfully.
Setting scheduler to Default
Generating T2I image for prompt: familyguy style, a lawyer arguing in court


  0%|          | 0/30 [00:00<?, ?it/s]

...

T2I image generated.
Setting scheduler to DDIM
Generating T2I image for prompt: batman in sky


  0%|          | 0/46 [00:00<?, ?it/s]

T2I image generated.
Setting scheduler to DDIM
Generating T2I image for prompt: duck in sky


  0%|          | 0/46 [00:00<?, ?it/s]

T2I image generated.
